### finns intro
Kan det vara en kandidat för WD beskriven av

In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2023-03-06 08:37:55.527086


In [8]:
import urllib3, json
import pandas as pd 
http = urllib3.PoolManager() 

url= "https://litteraturbanken.se/api/list_all/author?filter_and={%22_exists%22:%22intro%22}&to=10000"  

pd.set_option("display.max.columns", None) 
 
r = http.request('GET', url)
data = json.loads(r.data)
dfIntro = pd.json_normalize(data["data"])

In [10]:
dfIntro.head()

,authorid,authorid_norm,db_checksum,db_timestamp,db_timestamp_updated,doc_type,full_name,gender,imported,intro,intro_text,librisid,name_for_index,pictureinfo,popularity,searchable,show,surname,updated,birth.date,birth.plain,death.plain,wikidata.birthplace,wikidata.birthplace_label,wikidata.deathplace,wikidata.deathplace_label,wikidata.image,wikidata.sbl_link,wikidata.skbl_link,wikidata.sol_link,wikidata.wikidata_id,wikidata.wikipedia,sources,other_name,intro_author,intro_author_norm,death.date,picture,pseudonym,presentation,dramawebben.intro,dramawebben.intro_author,dramawebben.intro_author_norm,dramawebben.legacy_url,bibliography,external_ref,dramawebben.picture,dramawebben.picture_info,dramawebben.sources,seemore
0,AbeniusM,AbeniusM,76499012e7c07728be855a7b41a14baf,1675080014912,1.675080e+12,author,Margit Abenius,female,2017-04-14,\n<p>MARGIT ABENIUS (1899–1970): ”Margit Abeni...,\nMARGIT ABENIUS (1899–1970): ”Margit Abenius ...,ljx00mt45v0dfx5,"Abenius, Margit",None,2519.0,True,True,Abenius,2019-10-16,1899,1899,1970,"[14.614527777, 61.12]",Orsa församling,"[17.721805555, 60.011805555]",Lena Parish,http://commons.wikimedia.org/wiki/Special:File...,None,MargitAbenius,None,Q4933592,https://sv.wikipedia.org/wiki/Margit_Abenius,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BiörnerEJ,BiornerEJ,4b5b699934e910d889595e8dd4559a97,1675080015100,1.675080e+12,author,Eric Julius Biörner,male,2015-02-18,\n<p>ERIC JULIUS BIÖRNER eller BJÖRNER (1696–1...,\nERIC JULIUS BIÖRNER eller BJÖRNER (1696–1750...,c9prvz6w1sq9xjg,"Biörner, Eric Julius",None,3885.0,False,True,Biörner,2019-10-16,1696,1696,1750,"[17.315361111, 62.476277777]",Timrå församling,None,None,None,18253,None,None,Q5578306,https://sv.wikipedia.org/wiki/Eric_Julius_Bi%C...,"[Bernhard Meijer, <a href=""/forfattare/MeijerB...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DahlbergE,DahlbergE,a5bae1b55e7a7133d174724ea602fdb4,1675080015445,1.675080e+12,author,Erik Dahlbergh,male,2014-02-11,"\n<p>ERIK DAHLBERGH (1625–1703): ”grefve, topo...","\nERIK DAHLBERGH (1625–1703): ”grefve, topogra...",fcrttqfz2slp120,"Dahlbergh, Erik",None,23946.0,True,True,Dahlbergh,2019-10-16,1625,1625,1703,"[18.068611111, 59.329444444]",Stockholm,"[18.068611111, 59.329444444]",Stockholm,http://commons.wikimedia.org/wiki/Special:File...,15793,None,None,Q434533,https://sv.wikipedia.org/wiki/Erik_Dahlbergh,"[Bernhard Meijer, <a href=""/forfattare/MeijerB...",[Erik Dahlberg],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JohannesMagnus,JohannesMagnus,ecc878f2f7266d32924f21b79eeff4df,1675080016276,1.675080e+12,author,Johannes Magnus,male,2012-12-13,\n\n\n <p>JOHANNES MAGNUS (1488–1544) blev ...,\n\n\n JOHANNES MAGNUS (1488–1544) blev kan...,pm134g7756kd6t3,Johannes Magnus,None,67183.0,True,True,Johannes Magnus,2019-10-16,1488,1488,1544,"[15.625656134, 58.409418037]",Linköping,"[12.482777777, 41.893055555]",Rome,http://commons.wikimedia.org/wiki/Special:File...,12115,None,None,Q550662,https://sv.wikipedia.org/wiki/Johannes_Magnus,[<i>Svenskt biografiskt lexikon</i>],NaN,LotassL,LotassL,1544,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Matthias,Matthias,f6efc507203e9e834915e0aefb4ecf0c,1675080015577,1.675080e+12,author,Matthias Lincopensis,male,2016-09-09,"\n<p>MATTHIAS LINCOPENSIS (d. 1350), magister ...","\nMATTHIAS LINCOPENSIS (d. 1350), magister Mat...",nl023f864h172g5,Matthias Lincopensis,None,5194.0,True,True,Matthias Lincopensis,2019-10-16,1300,0000,1350,None,None,"[18.070833333, 59.325]",Stockholm City,None,9164,None,None,Q16649933,https://sv.wikipedia.org/wiki/Mathias_Ouidi,[<i>Svenskt biografiskt lexikon</i>],NaN,MiocevicL,MiocevicL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# WD query to get people Litteraturbanken and described by
endpoint_url = "https://query.wikidata.org/sparql"
import sys
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

# query people connected to Litteraturebank with no picture and born before 1920
# https://w.wiki/6Qhm
queryLittDesc = """#title: Beskriven av
SELECT (REPLACE(STR(?item), ".*Q", "Q") AS ?WikidataID) (CONCAT(?itemLabel, " ", STR(YEAR(?birthDayWD))) AS ?search) ?birthDayWD ?firstnameLabel ?lastnameLabel ?item 
?itemLabel (STR(YEAR(?birthDayWD)) AS ?BirthYearWD) ?authorid WHERE {
  ?item wdt:P5101 ?authorid;
    wdt:P1343 wd:Q10567910.
  OPTIONAL { ?item wdt:P735 ?firstname. }
  OPTIONAL { ?item wdt:P734 ?lastname. }
  OPTIONAL { ?item wdt:P569 ?birthDayWD. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv", "en". }
}"""



def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



WDLittDescdf = get_sparql_dataframe(endpoint_url, queryLittDesc)
WDLittDescdf.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   WikidataID      137 non-null    object
 1   search          136 non-null    object
 2   birthDayWD      136 non-null    object
 3   firstnameLabel  133 non-null    object
 4   lastnameLabel   122 non-null    object
 5   item            137 non-null    object
 6   itemLabel       137 non-null    object
 7   BirthYearWD     136 non-null    object
 8   authorid        137 non-null    object
dtypes: object(9)
memory usage: 9.8+ KB


In [15]:
WDLittbank_WD_merge =  pd.merge(dfIntro, WDLittDescdf, on='authorid',how="outer",indicator=True)
WDLittbank_WD_merge.rename(columns={"_merge": "WD_Littbank_merge_intro"},inplace = True)

WDLittbank_WD_merge["WD_Littbank_merge_intro"].value_counts()

left_only     629
both          130
right_only      7
Name: WD_Littbank_merge_intro, dtype: int64

In [24]:
columnsShow = ['authorid', 'WikidataID','WD_Littbank_merge_intro','intro'] 
WDLittbank_WD_merge[columnsShow]

,authorid,WikidataID,WD_Littbank_merge_intro,intro
0,AbeniusM,NaN,left_only,\n<p>MARGIT ABENIUS (1899–1970): ”Margit Abeni...
1,BiörnerEJ,NaN,left_only,\n<p>ERIC JULIUS BIÖRNER eller BJÖRNER (1696–1...
2,DahlbergE,NaN,left_only,"\n<p>ERIK DAHLBERGH (1625–1703): ”grefve, topo..."
3,JohannesMagnus,NaN,left_only,\n\n\n <p>JOHANNES MAGNUS (1488–1544) blev ...
4,Matthias,NaN,left_only,"\n<p>MATTHIAS LINCOPENSIS (d. 1350), magister ..."
...,...,...,...,...
761,NermanT,Q538109,right_only,NaN
762,WärnlöfAL,Q561698,right_only,NaN
763,OlofssonH,Q114121959,right_only,NaN
764,LjungbergPA,Q115945623,right_only,NaN


In [25]:
from IPython.display import HTML

WDLittbank_WD_merge["Littbank"] = "<a target=_blank href='https://litteraturbanken.se/f%C3%B6rfattare/" + \
WDLittbank_WD_merge["authorid"] + "'>link Littbank</a>"

WDLittbank_WD_merge["WD"] = "<a target=_blank href='https://www.wikidata.org/wiki/" + \
             WDLittbank_WD_merge["WikidataID"] + "'>" + WDLittbank_WD_merge["WikidataID"] + "</a>"
HTML(WDLittbank_WD_merge[["authorid","Littbank", "WikidataID","WD","intro"]
                                 ].to_html(escape=False))


,authorid,Littbank,WikidataID,WD,intro
0,AbeniusM,link Littbank,NaN,NaN,"\nMARGIT ABENIUS (1899–1970): ”Margit Abenius var en framstående litteraturforskare och en av sin tids viktigaste kritikerröster från tidigt 1930-tal och fram till sin död. Hon var även verksam som essäist, översättare och kulturförmedlare samt medlem i Samfundet De Nio.\n[…]\n\nHennes arbete med Karin Boyes litterära eftermäle inleddes redan 1942 med den tillsammans med Olof Lagercrantz redigerade antologin Karin Boye: minnen och studier. Margit Abenius redigerade och kommenterade även Karin Boyes Samlade skrifter, 1947–1949, detta under arbetet med sin Boyebiografi Drabbad av renhet: en bok om Karin Boyes liv och diktning, 1950. Biografin mottogs mycket positivt och den kom i decennier att påverka inte bara den allmänna bilden av Karin Boyes författarskap utan även synen på biografiskrivandet. Kritiska röster och polemik saknades dock inte och med tiden har inte minst Margit Abenius bild av Karin Boyes personlighet och hennes konservativa sätt att skriva om Karin Boyes homosexualitet rönt stark kritik.”\n\nCitaten ovan är hämtade ur Marta Ronnes artikel i Svenskt kvinnobiografiskt lexikon, läs vidare där: https://skbl.se/sv/artikel/MargitAbenius\n"
1,BiörnerEJ,link Littbank,NaN,NaN,"\nERIC JULIUS BIÖRNER eller BJÖRNER (1696–1750): ”fornforskare, f. 1696, blef 1725 sekreterare och 1738 assessor i antiqvitets-kollegium. Död 1750. Han utgaf en mängd arkeologiska skrifter och åtnjöt mycket länge stort anseende såväl utom som inom vårt land. Hans förnämsta arbeten, hvilka i hög grad bidrogo till att stärka nationalkänslan, voro följande: Nordiska kämpadater, i en sagoflock samlade, om forna kongar och hjältar (1737), Nordisk hjältaprydnad (1739) och Svea rikes hävdaålder (1748). Vidare må nämnas: De Suedia boreali (1717), Specimen hist. geogr. descriptionem Gothunheimiae o.s.v. exhibens (1721, fullst. 1741), Prodromus tractatuum de geographia Scandinaviae veteris & historiis gothicis o.s.v. (1726), Inledning till de yfverborna göters gamla häfder (1738) och Cogitationes critico-philol. de ortographia linguae suio-gothicae (1742).”\n\n\n\n"
2,DahlbergE,link Littbank,NaN,NaN,"\nERIK DAHLBERGH (1625–1703): ”grefve, topograf, f. 1625, d. 1703 som kungligt råd och fältmarskalk, är i literatur-historien bekant genom sitt stora planschverk Suecia antiqua et hodierna, som innehåller afbilder af (äfvensom af andra personer författade beskrifningar öfver) Sveriges förnämsta slott och städer m.m. Han skref för öfrigt en ’kort och sannfärdig berättelse’ om sitt eget lif (utg. 1757 i Gjörvells ’Svenska bibliotek’) och en dagbok (utg. 1823 af S. Lundblad). Se skrifter af A. Fryxell och K. G. Nordin.”\n\n\n\n"
3,JohannesMagnus,link Littbank,NaN,NaN,"\n\n\n JOHANNES MAGNUS (1488–1544) blev kanik i Linköping och Skara 1506. Någon tid därefter reste\n han till Louvain och sedermera till Köln för att fullborda sin utbildning, vilken han avslutade\n med ett teologie doktorat i Perugia (troligtvis 1520). Som Sten Sture d. y.:s ombud reste han\n till Rom där han av Hadrianus VI utnämndes till apostolisk nuntie i Sverige. Johannes Magnus\n främsta uppgift därvid var att bekämpa de spirande lutherska lärorna. Han återvände till\n Sverige 1523, blev vald till ärkebiskop, men kom att hamna i stridslinjen mellan katolska\n kyrkan och den allt starkare reformationen. Hans relationer med Gustav Vasa, som ville lösgöra\n Sverige från Rom, försämrades alltmer. Kungen sände honom till Polen under förevändning att\n han där skulle förhandla om det av kungen önskade äktenskapet med Sigismunds dotter Hedvig.\n Uppdraget syntes dock mer som en landsförvisning och Johannes Magnus skulle aldrig återse\n Sverige. Han fortsatte förgäves att verka för sitt hemlands återinlemmande i katolska kyrkan.\n Tillsammans med sin bror Olaus Magnus bosatte han sig 1538 i Venedig, varifrån de några år\n senare flyttade till Rom. Under sina sista år i livet ägnade sig Johannes Magnus åt ett\n omfattande skrifts